



> [chat history ,query]  ->input
\
> [sql_generation,general_qa, greeting , out of scope, default]



In [ ]:
agent_system_prompt="""You are a helpful AI Assistant that processes various type of user queries. your role is to throughly analyze the user query/question and classify them exactly one category and prepare them
for downstream processing without answering them unless they are user greetings, or can be answered from chat history.
for any query which is out of this scope, apologies to the user requesting him to ask relevant briefly providing him suggestion.

Definations of query_type:
1. sql_generation
select query_type_sql_generation if the question requests specific data points from our securities database, including :
- Financial metrics(price, market cap, price target price target date)
-Rating data(Rating text(Buy/Neutral/sell), rating type,rating Date)
-security status(Suspended, Restricted, coverage status)
-Company information (wire issuer, Name, comapny Name, RIS code)
-Research coverage details

2. classify as sql_generation if the:
-question is asking for specific values,lists, or comparisons
-query contains databse field names(comapny names/abbreviations) or related terms
-Requests for current /historical data points
-Filtering or sorting requests

Example for sql_generation:
-"what is Tesla's price target?"->sql_generation
-"List all suspended stocks"->sql_generation
-"which companies have buy ratings?"->sql_generation
-"show me stockes with market cap over 1000"->sql_generation
-"what is apple's current price?"->sql_generation
-"which stocks are restricted?"->sql_generation


2. general_qa
select query_type general_qa when the question seeks analysis, opinions or insights that require interpretation.

content includes;
-Investment analysis and expert opinions
-Market trends and economic commentary
-valuation explainations ans forecasts
-industry analysis, sector analysis ans research insights

classify as general_qa if the question:
  Asks why, how or what do you think
-seeks opinins view or analysis
-Discussion over market trends or economic impacts
-Requests explanations or interpretations
-any question on any industry or sector

example for general_qa:
-what is teh ABC's view on tesla's valuation?
-how does inflationimpact equity market?
-what are current investment themes?
-what is abc's view on it sector?
-how is mining sector performing?
-what factors drive market volatility?

3. direct
select query_type non_contextual when query doesn't fall into sql_generation or general_qa
-to answer to user's greeting including hi, hello, etc.
-to apologies to user when he is asking anything whihc is not related to above two categories like out-of-scope queries
(e.g. what's the weather?); strictly avoid answering to such questions
-to provide answer from chat_history or summarize the past conversation

4. Special gandling:
Mixed queries(data +opinion)
when the question combines factual data requests with opinion components, always choose sql_generation

examples:
what is ABC's view on tesla's current price target?->sql_generation
how does xyz view apple's market cap prospects?-> sql_generation

Ambiguous cases:
-if uncertain between the two, default to sql_generation
- only choose general_qa when clearly requesting analysis or opinions

note on response and formulated_question:
-for query_type 'sql_generation' and general_qaprovide a formulated_question (rephrased if needed or original if it clear and complete).
-Don't generate response/answers for sql_generation or general_qa queries :
only provide the formulated question for downstream nodes.

chat_history : {chat_history}
provide your output in json format.
"""

why are we forcing it to sql generation instead of seperate out question.

In [1]:
! git clone https://github.com/microsoft/PromptWizard

Cloning into 'PromptWizard'...
remote: Enumerating objects: 578, done.
remote: Counting objects: 100% (187/187), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 578 (delta 145), reused 124 (delta 124), pack-reused 391 (from 1)
Receiving objects: 100% (578/578), 1.68 MiB | 14.43 MiB/s, done.
Resolving deltas: 100% (291/291), done.


In [2]:
cd PromptWizard

/content/PromptWizard


In [3]:
!pip install -e .

Obtaining file:///content/PromptWizard
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of llama-index-indices-managed-llama-cloud to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-index-indices-managed-llama-cloud to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa

In [1]:
import openai
openai.__version__
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [2]:
os.environ["USE_OPENAI_API_KEY"] = "True"
os.environ["OPENAI_MODEL_NAME"] ="gpt-4o-mini"
os.environ["OPENAI_API_VERSION"] = "1.93.0"

In [3]:
import sys
sys.path.insert(0, "../../")
import promptwizard
from promptwizard.glue.promptopt.instantiate import GluePromptOpt
from promptwizard.glue.promptopt.techniques.common_logic import DatasetSpecificProcessing
from promptwizard.glue.common.utils.file import save_jsonlist
from typing import Any
from tqdm import tqdm
from re import compile, findall
import os
from datasets import load_dataset

In [4]:
import yaml
def update_yaml_file(file_path,config_dict):

    with open(file_path, 'r') as file:
        data = yaml.safe_load(file)


    for field,value in config_dict.items():
        data[field] = value

    with open(file_path, 'w') as file:
        yaml.dump(data, file, default_flow_style=False)

    print("YAML file updated successfully!")

In [5]:
# training_dataset=[
#     {"query": "what is Tesla's price target?", "query_type": "sql_generation","response":"", "formulated_question": ""},
# {"query": "List all suspended stocks", "query_type": "sql_generation", "response":"", "formulated_question": ""},
# {"query": "Which companies have buy ratings?", "query_type": "sql_generation", "response":"", "formulated_question": ""},
# {"query": "What is ABC's view on Tesla's valuation?", "query_type": "general_qa",  "response":"","formulated_question": ""},
# {"query": "How does inflation impact equity market?", "query_type": "general_qa", "response":"", "formulated_question": ""},
# {"query": "Hi", "query_type": "direct","response":"hi,how can i help you?", "formulated_question": ""},
# {"query": "What's the weather in Delhi?", "query_type": "direct","response":"out of scope", "formulated_question": ""},
# {"query": "How does XYZ view Apple’s market cap prospects?", "query_type": "sql_generation",  "response":"","formulated_question": ""},
# {"query": "Show me stocks with market cap over 1000", "query_type": "sql_generation", "response":"", "formulated_question": ""},
# {"query": "What are current investment themes?", "query_type": "general_qa","response":"", "formulated_question": ""}
# ]
training_dataset=[
    {"question": "what is Tesla's price target?", "final_answer": "sql_generation","response":""},
{"question": "List all suspended stocks", "final_answer": "sql_generation", "response":""},
{"question": "Which companies have buy ratings?", "final_answer": "sql_generation", "response":""},
{"question": "What is ABC's view on Tesla's valuation?", "final_answer": "general_qa",  "response":""},
{"question": "How does inflation impact equity market?", "final_answer": "general_qa", "response":""},
{"question": "Hi", "final_answer": "direct","response":"hi,how can i help you?"},
{"question": "What's the weather in Delhi?", "final_answer": "direct","response":"Hi, I’m sorry, but I can only answer questions that are related to our business model."},
{"question": "How does XYZ view Apple’s market cap prospects?", "final_answer": "sql_generation",  "response":""},
{"question": "Show me stocks with market cap over 1000", "final_answer": "sql_generation", "response":""},
{"question": "What are current investment themes?", "final_answer": "general_qa","response":""},
{"question": "can you tell me about donald trump", "final_answer": "direct","response":"Hi, I’m sorry, but I can only answer questions that are related to our business model."},
{"question": "What's the distance between kashmir and new delhi", "final_answer": "direct","response":"Hi, I’m sorry, but I can only answer questions that are related to our business model."},
{"question": "what is the fundamental law of nature", "final_answer": "direct","response":"Hi, I’m sorry, but I can only answer questions that are related to our business model."},
{"question": "give me top five dishes of belgium", "final_answer": "direct","response":"Hi, I’m sorry, but I can only answer questions that are related to our business model."},
]

In [6]:
import re
from typing import Any, List, Dict
from tqdm import tqdm

class CustomQueryClassifier(DatasetSpecificProcessing):
    VALID_LABELS = {"sql_generation", "general_qa", "direct"}

    def dataset_to_jsonl(self, dataset_jsonl: str, **kwargs: Any) -> None:
        def normalize_final_answer(answer: str) -> str:
            """
            Normalize the label to exact match within allowed labels.
            If the label is missing or invalid, return INVALID_ANS.
            """
            if not answer or not answer.strip():
                return self.INVALID_ANS

            cleaned = answer.strip()
            return cleaned if cleaned in self.VALID_LABELS else self.INVALID_ANS

        examples_set: List[Dict[str, Any]] = []

        for _, sample in tqdm(enumerate(kwargs["dataset"]), desc="Processing Query Classification Samples"):
            question = sample.get("query", "").strip()
            answer = sample.get("final_answer", "")

            example = {
                DatasetSpecificProcessing.QUESTION_LITERAL: question,
                DatasetSpecificProcessing.FINAL_ANSWER_LITERAL: normalize_final_answer(answer)
            }
            examples_set.append(example)

        save_jsonlist(dataset_jsonl, examples_set, "w")

    def extract_final_answer(self, answer: str) -> str:
        """
        Return cleaned final answer only if it's a valid label.
        """
        if not answer or not answer.strip():
            return self.INVALID_ANS

        cleaned = answer.strip()
        return cleaned if cleaned in self.VALID_LABELS else self.INVALID_ANS


In [7]:
query_type=CustomQueryClassifier()

In [8]:
import json
with open("training_data.jsonl", "w", encoding="utf-8") as f:
    for item in training_dataset:
        json.dump(item, f)
        f.write("\n")

In [9]:
task_desription="""You are an AI assitant that analyze the user query and classify them into teh following classes:-
i)sql_generation :- select query_type_sql_generation if the question requests specific data points from our securities database, including :
- Financial metrics(price, market cap, price target price target date)
-Rating data(Rating text(Buy/Neutral/sell), rating type,rating Date)
-security status(Suspended, Restricted, coverage status)
-Company information (wire issuer, Name, comapny Name, RIS code)
-Research coverage details
  classify as sql_generation if the:
-question is asking for specific values,lists, or comparisons
-query contains databse field names(comapny names/abbreviations) or related terms
-Requests for current /historical data points
-Filtering or sorting requests
ii)general_qa :-select query_type general_qa when the question seeks analysis, opinions or insights that require interpretation.
content includes;
-Investment analysis and expert opinions
-Market trends and economic commentary
-valuation explainations ans forecasts
-industry analysis, sector analysis ans research insights

classify as general_qa if the question:
  Asks why, how or what do you think
-seeks opinins view or analysis
-Discussion over market trends or economic impacts
-Requests explanations or interpretations
-any question on any industry or sector
iii)direct:-if that is greeting or some irrelevant question ,chat_history:- if the given query referring previous task/response if question is not refering to any of them then respond out of scope.

repone should be in output json
json{
  query:str=Field(description="the user query"),
  query_type:str=Field(description="the type of query ["sql_generation","general_qa","direct","chat_history"]),
  response:str=field(description="the resposne for direct query (greeting,summaries or answer from chat_history or out of scope) else empty for sql_generation and general_qa")
  formulated_query:str=field(description=the rephrased or original query if needed else empty)
}"""

>iii)non-contextual_q:-if that is greeting or some irrelevant question
/
> ["sql_generation","general_qa","non-contextual_q","chat_history"]


In [10]:
path_to_config = "PromptWizard/demos/scenarios/configs"
promptopt_config_path = os.path.join(path_to_config, "promptopt_config.yaml")
setup_config_path = os.path.join(path_to_config, "setup_config.yaml")

file_path = "PromptWizard/demos/scenarios/configs/promptopt_config.yaml"
config_dict = {
                "task_description": task_desription,
                "base_instruction": """Lets think step by step and Special handling:
                      Mixed queries(data +opinion)
                      when the question combines factual data requests with opinion components, always choose sql_generation
                      Ambiguous cases:
                      -if uncertain between the two, default to sql_generation
                      - only choose general_qa when clearly requesting analysis or opinions.
                      and response should be in  required output json only.""",
                "mutation_rounds": 5,
                "few_shot_count": 2,
                "generate_reasoning": True,
                "mutate_refine_iterations" : 3,
                "seen_set_size":6
               }
update_yaml_file(file_path,config_dict)

YAML file updated successfully!


In [11]:
gp = GluePromptOpt(promptopt_config_path,
                   setup_config_path,
                   dataset_jsonl = os.path.join("/content/","training_data.jsonl"),
                   data_processor = query_type)

Setup configurations parameters: [('assistant_llm', AssistantLLM(prompt_opt='gpt-4o')), ('description', None), ('dir_info', Dir(base_dir='logs', log_dir_name='glue_logs')), ('experiment_name', 'gsm8k'), ('mode', 'offline')] 


Prompt Optimization parameters: [('answer_format', 'For each question present the reasoning followed by the correct answer.'), ('base_instruction', 'Lets think step by step and Special handling:\n                      Mixed queries(data +opinion)\n                      when the question combines factual data requests with opinion components, always choose sql_generation\n                      Ambiguous cases:\n                      -if uncertain between the two, default to sql_generation\n                      - only choose general_qa when clearly requesting analysis or opinions.\n                      and response should be in  required output json only.'), ('few_shot_count', 2), ('generate_expert_identity', True), ('generate_intent_keywords', False), ('generate

In [12]:
best_prompt, expert_profile = gp.get_best_prompt(use_examples=True,run_without_train_examples=False,generate_synthetic_examples=False)


Mutating Task Description....


Iterations completed:   0%|          | 0/3 [00:00<?, ?it/s]
 + Starting iteration: 1 
 current_base_instruction: Lets think step by step and Special handling:
                      Mixed queries(data +opinion)
                      when the question combines factual data requests with opinion components, always choose sql_generation
                      Ambiguous cases:
                      -if uncertain between the two, default to sql_generation
                      - only choose general_qa when clearly requesting analysis or opinions.
                      and response should be in  required output json only.
mutation_round=0 mutated_sample_prompt=You are given a task description and a prompt instruction and different styles known as meta prompts:
[Task Description]: You are an AI assitant that analyze the user query and classify them into teh following classes:-
i)sql_generation :- select query_type_sql_generation if the question requests specific data points from our securities 

dataset_subset [{'question': "what is Tesla's price target?", 'final_answer': 'sql_generation', 'response': ''}]
critique_example_set, correct_count
[{'question': "what is Tesla's price target?", 'final_answer': 'sql_generation', 'response': ''}] 0
Loop completed
dataset_subset [{'question': 'List all suspended stocks', 'final_answer': 'sql_generation', 'response': ''}]
critique_example_set, correct_count
[{'question': 'List all suspended stocks', 'final_answer': 'sql_generation', 'response': ''}] 0
Loop completed
dataset_subset [{'question': 'List all suspended stocks', 'final_answer': 'sql_generation', 'response': ''}]
critique_example_set, correct_count
[{'question': 'List all suspended stocks', 'final_answer': 'sql_generation', 'response': ''}] 0
Loop completed
dataset_subset [{'question': 'Hi', 'final_answer': 'direct', 'response': 'hi,how can i help you?'}]
critique_example_set, correct_count
[{'question': 'Hi', 'final_answer': 'direct', 'response': 'hi,how can i help you?'}] 0
L

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
RuntimeError: reentrant call inside <_io.BufferedWriter name='/content/logs/gsm8k/glue_logs/glue_logs.log'>
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/

dataset_subset [{'question': 'List all suspended stocks', 'final_answer': 'sql_generation', 'response': ''}]
critique_example_set, correct_count
[{'question': 'List all suspended stocks', 'final_answer': 'sql_generation', 'response': ''}] 0
Loop completed
dataset_subset [{'question': "what is Tesla's price target?", 'final_answer': 'sql_generation', 'response': ''}]
critique_example_set, correct_count
[{'question': "what is Tesla's price target?", 'final_answer': 'sql_generation', 'response': ''}] 0
Loop completed
dataset_subset [{'question': 'Which companies have buy ratings?', 'final_answer': 'sql_generation', 'response': ''}]
critique_example_set, correct_count
[{'question': 'Which companies have buy ratings?', 'final_answer': 'sql_generation', 'response': ''}] 0
Loop completed
dataset_subset [{'question': 'Which companies have buy ratings?', 'final_answer': 'sql_generation', 'response': ''}]
critique_example_set, correct_count
[{'question': 'Which companies have buy ratings?', 'fin

...

100%|██████████| 3/3 [00:36<00:00, 12.03s/it]



Generating CoT Reasoning for In-Context Examples....


100%|██████████| 2/2 [00:09<00:00,  4.51s/it]



Generating Expert Identity....


Expert Identity: You are a highly skilled AI analyst with expertise in natural language processing and query classification. Your training has equipped you with the ability to understand and interpret user queries effectively. You can accurately classify queries into specific categories based on their content and intent. With a strong grasp of database terminology and the nuances of financial analysis, you can distinguish between requests for data generation, general inquiries, and irrelevant or conversational prompts. Your analytical capabilities allow you to formulate precise responses, ensuring that each query is categorized correctly as either `sql_generation`, `general_qa`, `direct`, or `chat_history`. You are adept at rephrasing queries when necessary and providing appropriate responses based on the classification, making you an invaluable resource for users seeking information or insights. Your attention to detail and understanding of context enable you to deliver accurate and r

In [13]:
print(best_prompt)


You are an AI assistant that analyzes user queries and classifies them into the following categories:

i) **sql_generation**: Select `query_type_sql_generation` if the question requests specific data points from our securities database. This includes:
- Financial metrics (e.g., price, market cap, price target, price target date)
- Rating data (e.g., rating text such as Buy/Neutral/Sell, rating type, rating date)
- Security status (e.g., suspended, restricted, coverage status)
- Company information (e.g., wire issuer, name, company name, RIS code)
- Research coverage details

Classify as `sql_generation` if the:
- Question asks for specific values, lists, or comparisons.
- Query contains database field names (e.g., company names/abbreviations) or related terms.
- Requests for current or historical data points, including specific time frames (e.g., "Q4 2023").
- Includes filtering or sorting requests, such as thresholds (e.g., "market cap over $10 billion").
- Involves multiple data poi

In [14]:
print(expert_profile)

You are a highly skilled AI analyst with expertise in natural language processing and query classification. Your training has equipped you with the ability to understand and interpret user queries effectively. You can accurately classify queries into specific categories based on their content and intent. With a strong grasp of database terminology and the nuances of financial analysis, you can distinguish between requests for data generation, general inquiries, and irrelevant or conversational prompts. Your analytical capabilities allow you to formulate precise responses, ensuring that each query is categorized correctly as either `sql_generation`, `general_qa`, `direct`, or `chat_history`. You are adept at rephrasing queries when necessary and providing appropriate responses based on the classification, making you an invaluable resource for users seeking information or insights. Your attention to detail and understanding of context enable you to deliver accurate and relevant results e

In [15]:
expert_profile

'You are a highly skilled AI analyst with expertise in natural language processing and query classification. Your training has equipped you with the ability to understand and interpret user queries effectively. You can accurately classify queries into specific categories based on their content and intent. With a strong grasp of database terminology and the nuances of financial analysis, you can distinguish between requests for data generation, general inquiries, and irrelevant or conversational prompts. Your analytical capabilities allow you to formulate precise responses, ensuring that each query is categorized correctly as either `sql_generation`, `general_qa`, `direct`, or `chat_history`. You are adept at rephrasing queries when necessary and providing appropriate responses based on the classification, making you an invaluable resource for users seeking information or insights. Your attention to detail and understanding of context enable you to deliver accurate and relevant results 